<a href="https://colab.research.google.com/github/KOSETTPAI/Internship/blob/main/Simbolo_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
from datasets import load_dataset_builder, load_dataset

In [2]:
def create_break_pattern():
    """Creates and returns the regular expression pattern for Myanmar syllable breaking."""
    my_consonant = r"က-အ"
    en_char = r"a-zA-Z0-9"
    other_char = r"ဣဤဥဦဧဩဪဿ၌၍၏၀-၉၊။!-/:-@[-`{-~\s"
    subscript_symbol = r'္'
    a_that = r'်'

    # Regular expression pattern for Myanmar syllable breaking
    return re.compile(
        r"((?<!" + subscript_symbol + r")[" + my_consonant + r"]"
        r"(?!["
        + a_that + subscript_symbol + r"])"
        + r"|[" + en_char + other_char + r"])"
    )

In [3]:
def break_syllables(line, break_pattern, separator):
    """Applies syllable breaking rules to a line."""
    line = re.sub(r'\s+', ' ', line.strip())
    segmented_line = break_pattern.sub(separator + r"\1", line)

    # Remove the leading delimiter if it exists
    if segmented_line.startswith(separator):
        segmented_line = segmented_line[len(separator):]

    # Replace delimiter+space+delimiter with a single space
    double_delimiter = separator + " " + separator
    segmented_line = segmented_line.replace(double_delimiter, " ")

    return segmented_line

In [4]:
simbolo = dataset = load_dataset("Simbolo-Servicio/wiki-burmese-sentences")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
sim_train = simbolo['train']['text']

In [6]:
EOS = []

break_pattern = create_break_pattern()

for sentence in sim_train:
  EOS.append(break_syllables(sentence, break_pattern, " ").split()[-1])

In [7]:
EOS_unique = list(set(EOS))

In [8]:
EOS_frequencies = {i: EOS.count(i) for i in EOS_unique}

In [9]:
EOS_frequencies

{'ရှည်': 4,
 'ချူ': 1,
 'ခြာ': 1,
 'ကိုက်': 1,
 'စုံ': 1,
 'နည်း”': 3,
 '8': 3,
 'မွေး': 3,
 'တပ်': 3,
 'တိုး': 8,
 'သွတ်': 2,
 'မား': 1,
 'ချည်': 1,
 'မည်”': 1,
 'ဝံ': 2,
 'ပြတ်': 3,
 'သေည်': 3,
 'တစ်': 1,
 'ချိုး': 2,
 'ဖယ်': 1,
 'တတ်': 31,
 '3': 2,
 'အေး': 3,
 'သျို': 1,
 '4': 3,
 'မေး': 2,
 'လာန်': 4,
 'တွေ့': 4,
 'ညာ': 16,
 'ရုံ': 6,
 'ကောင်': 4,
 'z': 1,
 'ယူ': 9,
 'ခြင်း': 430,
 'လတ္တံ့': 16,
 'ငူ': 6,
 'သိ': 15,
 'တောက်': 3,
 'ရိုက်': 1,
 'ဘက်': 5,
 'ကင်း': 3,
 'ဍ': 1,
 'ဖြုံ': 3,
 'မြိန်': 1,
 'ဠီ': 1,
 'နေ': 4,
 'ကပ်': 1,
 'ကုတ်': 2,
 ':': 2,
 'ရစ်': 1,
 'ချွန်း': 1,
 'ဘော': 12,
 'ဝတ္တံ': 2,
 'ခွ': 4,
 'ကယ်': 1,
 'မည်း': 1,
 'ခြည်း': 1,
 'ကုန်': 35,
 'သံ': 4,
 'ကျော်': 5,
 'ဘွတ်ခ်': 2,
 '9': 3,
 'မန်': 3,
 'မြွေ': 1,
 'ကွာ': 1,
 'မျက်': 1,
 'အတ္ထုပ္ပတ္တိ': 1,
 'ကိုင်': 3,
 'အိတ်': 1,
 'ပြေး': 1,
 'နယ်': 10,
 'နိ': 4,
 'မန်း': 2,
 'ပါး': 43,
 'ဆောင်း': 4,
 'ထိုး': 4,
 'ဖိုး': 3,
 'တော်': 48,
 'ယဉ်': 36,
 'နှစ်': 19,
 'သ': 8,
 'စွမ်း': 1,
 'နှာ': 4,
 'ဒတ်': 11,
 'မြည်': 1,
 'ကျ